# Transfer Learning experiments

In [49]:
import os
import torch
import mlflow
import numpy as np
from torch import nn
from torch import optim
from collections import OrderedDict
import torch.nn.functional as F
from torchvision import datasets, transforms, models

## Transfer Learning with Resnet 50

### Getting Resnet model

In [50]:
mdl = models.resnet50(pretrained=True)

# Freezing the paramiters of the layers we do not want to train
for parameters in mdl.parameters():
    parameters.requires_grad = False

In [51]:
# Updating Classification layer 
_inputs = mdl.fc.in_features

mdl.fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(_inputs, 500)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(0.2)),
    ('fc2', nn.Linear(500, 39)),
    ('output', nn.LogSoftmax(dim=1))
]))


In [52]:
def train(model, train_loader, validation_loader, config, n_epochs=10, stopping_treshold=None):

    if torch.cuda.is_available():
        print('CUDA is available!  Training on GPU ...')
        model.cuda()


    # Loss and optimizer setup 
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

    # Setting minimum validation loss to inf
    validation_loss_minimum = np.Inf 
    train_loss_history = []
    validation_loss_history = []

    for epoch in range(1, n_epochs +1):

        training_loss = 0.0
        validation_loss = 0.0

        # Training loop
        training_accuracies = []
        for X, y in train_loader:
            
            # Moving data to gpu if using 
            if torch.cuda.is_available():
                X, y = X.cuda(), y.cuda()
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # calculate the batch loss
            loss = criterion(output, y)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            training_loss += loss.item()*X.size(0)

            # calculating accuracy
            ps = torch.exp(output)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == y.view(*top_class.shape)
            training_accuracies.append(torch.mean(equals.type(torch.FloatTensor)).item())

        # Validation Loop
        with torch.no_grad():
            accuracies = []
            for X, y in validation_loader:

                # Moving data to gpu if using 
                if torch.cuda.is_available():
                    X, y = X.cuda(), y.cuda()
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(X)
                # calculate the batch loss
                loss = criterion(output, y)
                # update validation loss
                validation_loss += loss.item()*X.size(0)

                # calculating accuracy
                ps = torch.exp(output)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == y.view(*top_class.shape)
                accuracies.append(torch.mean(equals.type(torch.FloatTensor)).item())
                
        # Mean loss 
        mean_training_loss = training_loss/len(train_loader.sampler)
        mean_validation_loss = validation_loss/len(validation_loader.sampler)
        mean_train_accuracy = sum(training_accuracies)/len(training_accuracies)
        mean_accuracy = sum(accuracies)/len(accuracies)
        train_loss_history.append(mean_training_loss)
        validation_loss_history.append(mean_validation_loss)

        # Printing epoch stats
        print(f'Epoch: {epoch}/{n_epochs}, ' +\
              f'Training Loss: {mean_training_loss:.3f}, '+\
              f'Train accuracy {mean_train_accuracy:.3f} ' +\
              f'Validation Loss: {mean_validation_loss:.3f}, '+\
              f'Validation accuracy {mean_accuracy:.3f}')

        # logging with mlflow 
        if mlflow.active_run():
            mlflow.log_metric('loss', mean_training_loss, step=epoch)
            mlflow.log_metric('accuracy', mean_train_accuracy, step=epoch)
            mlflow.log_metric('validation_accuracy', mean_accuracy, step=epoch)
            mlflow.log_metric('validation_loss', mean_validation_loss, step=epoch)

        # Testing for early stopping
        if stopping_treshold:
            if mean_validation_loss < validation_loss_minimum:
                validation_loss_minimum = mean_validation_loss
            elif len([v for v in validation_loss_history[-stopping_treshold:] if v >= validation_loss_minimum]) >= stopping_treshold:
                print(f"Stopping early at epoch: {epoch}/{n_epochs}")
                break
        

    return train_loss_history, validation_loss_history

### Training - 32x32 resolution 

#### Loading data 

In [53]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(32),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(34),
                                      transforms.CenterCrop(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                    ])
# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_32')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)


In [54]:
# Configs 
config = {
    'max_epochs': 200,
    'learning_rate': 0.002,
    'resolution': 32
}

In [55]:

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

mlflow.set_experiment("Plant Leaf Disease")

with mlflow.start_run():
    mlflow.log_param('framework', 'pytorch')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', 'Resnet50')
    mlflow.log_params(config)
    train(mdl, train_loader, validation_loader, config, n_epochs=config['max_epochs'], stopping_treshold=15)

CUDA is available!  Training on GPU ...
Epoch: 1/200, Training Loss: 2.526, Train accuracy 0.325 Validation Loss: 2.193, Validation accuracy 0.394
Epoch: 2/200, Training Loss: 2.288, Train accuracy 0.375 Validation Loss: 2.070, Validation accuracy 0.428
Epoch: 3/200, Training Loss: 2.235, Train accuracy 0.387 Validation Loss: 2.003, Validation accuracy 0.439
Epoch: 4/200, Training Loss: 2.207, Train accuracy 0.393 Validation Loss: 1.987, Validation accuracy 0.437
Epoch: 5/200, Training Loss: 2.173, Train accuracy 0.399 Validation Loss: 1.993, Validation accuracy 0.450
Epoch: 6/200, Training Loss: 2.152, Train accuracy 0.405 Validation Loss: 1.958, Validation accuracy 0.445
Epoch: 7/200, Training Loss: 2.139, Train accuracy 0.410 Validation Loss: 1.940, Validation accuracy 0.451
Epoch: 8/200, Training Loss: 2.128, Train accuracy 0.410 Validation Loss: 1.927, Validation accuracy 0.456
Epoch: 9/200, Training Loss: 2.115, Train accuracy 0.414 Validation Loss: 1.897, Validation accuracy 0.4

### Training 224X224 resolution

In [56]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                    ])

# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_224')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)

In [57]:
mdl = models.resnet50(pretrained=True)

# Freezing the paramiters of the layers we do not want to train
for parameters in mdl.parameters():
    parameters.requires_grad = False

# Updating Classification layer 
_inputs = mdl.fc.in_features

mdl.fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(_inputs, 500)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(0.2)),
    ('fc2', nn.Linear(500, 39)),
    ('output', nn.LogSoftmax(dim=1))
]))

# Configs 
config = {
    'max_epochs': 200,
    'learning_rate': 0.002,
    'resolution': 224
}

In [58]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

mlflow.set_experiment("Plant Leaf Disease")

with mlflow.start_run():
    mlflow.log_param('framework', 'pytorch')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', 'Resnet50')
    mlflow.log_params(config)
    train(mdl, train_loader, validation_loader, config, n_epochs=config['max_epochs'], stopping_treshold=15)

CUDA is available!  Training on GPU ...
Epoch: 1/200, Training Loss: 0.991, Train accuracy 0.708 Validation Loss: 0.536, Validation accuracy 0.827
Epoch: 2/200, Training Loss: 0.697, Train accuracy 0.785 Validation Loss: 0.372, Validation accuracy 0.877
Epoch: 3/200, Training Loss: 0.653, Train accuracy 0.797 Validation Loss: 0.355, Validation accuracy 0.879
Epoch: 4/200, Training Loss: 0.625, Train accuracy 0.806 Validation Loss: 0.329, Validation accuracy 0.890
Epoch: 5/200, Training Loss: 0.606, Train accuracy 0.813 Validation Loss: 0.326, Validation accuracy 0.891
Epoch: 6/200, Training Loss: 0.594, Train accuracy 0.817 Validation Loss: 0.286, Validation accuracy 0.904
Epoch: 7/200, Training Loss: 0.579, Train accuracy 0.822 Validation Loss: 0.301, Validation accuracy 0.899
Epoch: 8/200, Training Loss: 0.566, Train accuracy 0.825 Validation Loss: 0.316, Validation accuracy 0.891
Epoch: 9/200, Training Loss: 0.558, Train accuracy 0.827 Validation Loss: 0.285, Validation accuracy 0.8